In [ ]:
# Set up the python environment
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# The Electric Dipole

We will explore the electric dipole, which consists of two equal but opposite charges separated by a distance $l$.  We plot the full potential (using the sum of the simple electrostatic formulae for the individual charges) and then the approximate potential as derived in the notes using a first order expansion.

To fit with the notes (and the normal choice of axes) we are assuming that we work in the $x-z$ plane.

We are implicitly working in atomic units, where $1/4\pi\epsilon_0 = 1$ and the charge on an electron is $-1$.

In [ ]:
# Set up the simulation space: box size and number of grid points
Lx = 10
Ly = 10
npoints = 201

# 2D numpy arrays in Cartesian and spherical polar coordinates
linear_extent = np.linspace(-Lx,Lx,npoints)
x_2d, z_2d = np.meshgrid(linear_extent, linear_extent)
r_2d = np.sqrt(x_2d**2 + z_2d**2)
theta_2d = np.arctan2(x_2d,z_2d)

# Dipole size and related parameters
# For near field set l close to Ly; for far field make it small
# This shows fields near the dipole (where the approximation is poor)
# dipole_l = 8.1
# This shows fields far from the dipole (approximation is good)
dipole_l = 1.1

# Limit on potential plotting
vlim = 0.1

# Values for contours: adjust depending on dipole size
# Good for near field
#contour_values = [-.4,-.2,-.1,-0.05,-0.025,-0.0125,0.0125,0.025,0.05,.1,.2,.4]
# Good for far field
contour_values = [-4,-2,-1,-0.5,-0.25,-0.125,0.125,0.25,0.5,1,2,4]

In [ ]:
# Position the dipole along the z axis (assume that we work with y=0)
pos_p = np.array([0,0+dipole_l/2])
pos_m = np.array([0,0-dipole_l/2])

# Arrays containing the distance between grid point and positive and negative charges
r_p = np.sqrt((x_2d-pos_p[0])**2 + (z_2d-pos_p[1])**2)
r_m = np.sqrt((x_2d-pos_m[0])**2 + (z_2d-pos_m[1])**2)

# Charges
qp = 1
qm = -1

# Find potential using simple electrostatics
pot_p = qp*np.divide(1.0,r_p,out=np.zeros_like(theta_2d),where=r_p>1e-8)
pot_m = qm*np.divide(1.0,r_m,out=np.zeros_like(theta_2d),where=r_m>1e-8)
pot = pot_p + pot_m

# NB Use np.divide above to avoid divide by zero errors

In [ ]:
# Plot potential - choose origin consistent with extent
plt.imshow(pot,vmin=-vlim,vmax=vlim,extent=(-10,10,-10,10),origin='lower')
plt.colorbar(label="Electrostatic potential")

plt.contour(x_2d,z_2d,pot,contour_values,colors='w')
plt.plot(pos_p[0],pos_p[1],'ro')
plt.plot(pos_m[0],pos_m[1],'bo')
plt.title("Exact potential from dipole")
plt.savefig("EDipPot.png",dpi=300)

## Approximate potential

The approximate potential is given by:

$$\frac{ql\cos\theta}{r^2}$$

We use the `np.divide` function below to avoid a divide by zero.  The visual appearance of the two potentials is essentially identical in the far field (say $l=0.1$) but is clearly different in the near field ($l=8.1$).

In [ ]:
# Avoid divide by zero by using np.divide function
pot_approx = np.divide(qp*dipole_l*np.cos(theta_2d),r_2d**2,out=np.zeros_like(theta_2d),where=r_2d>1e-8)

# Plot potential
plt.imshow(pot_approx,vmin=-vlim,vmax=vlim,extent=(-10,10,-10,10),origin='lower')
plt.colorbar()
plt.contour(x_2d,z_2d,pot_approx,contour_values,colors='w')
plt.plot(pos_p[0],pos_p[1],'ro')
plt.plot(pos_m[0],pos_m[1],'bo')

## Difference in potential

We now plot the difference between the approximate and full potentials.  If we are in the regime where the approximation was derived ($r\gg l$) then the difference is small, and confined to the area near the dipole.

In [ ]:
plt.imshow(pot_approx-pot,vmin=-vlim,vmax=vlim,extent=(-10,10,-10,10),origin='lower')
plt.colorbar()
plt.contour(x_2d,z_2d,pot_approx-pot,contour_values,colors='w')
plt.plot(pos_p[0],pos_p[1],'ro')
plt.plot(pos_m[0],pos_m[1],'bo')
plt.title("Difference between approximate and full dipole potentials")

## Exact field

We can plot the exact field, also using simple electrostatics.  Plotting field lines can be more challenging: we include the potential plotted in colour as background.

In [ ]:
# Simple electrostatics for positive and negative charges
field_p_x = qp*np.divide(x_2d - pos_p[0],r_p**2,out=np.zeros_like(theta_2d),where=r_p>1e-8)
field_p_z = qp*np.divide(z_2d - pos_p[1],r_p**2,out=np.zeros_like(theta_2d),where=r_p>1e-8)
field_m_x = qm*np.divide(x_2d - pos_m[0],r_m**2,out=np.zeros_like(theta_2d),where=r_m>1e-8)
field_m_z = qm*np.divide(z_2d - pos_m[1],r_m**2,out=np.zeros_like(theta_2d),where=r_m>1e-8)

# Dipole field by summing charge fields
field_exact_x = field_p_x + field_m_x
field_exact_z = field_p_z + field_m_z

# Potential as background
plt.imshow(pot,vmin=-vlim,vmax=vlim,extent=(-10,10,-10,10),origin='lower')
plt.colorbar(label="Electrostatic potential")
plt.plot(pos_p[0],pos_p[1],'ro')
plt.plot(pos_m[0],pos_m[1],'bo')
plt.streamplot(x_2d,z_2d,field_exact_x,field_exact_z,density=1.0,color='white')
plt.axis('scaled')
plt.title("Electric field for dipole")
plt.savefig("EDipField.png",dpi=300)

## Approximate field

The approximate field can be written in spherical polars as:
$$E_r = 2\frac{l\cos\theta}{r^3}$$
$$E_\theta = \frac{l\sin\theta}{r^3}$$
We then convert to Cartesian coordinates

In [ ]:
# Calculate approximate field in spherical polar coordinates
field_r_approx = 2*np.divide(dipole_l*np.cos(theta_2d),r_2d**3,out=np.zeros_like(theta_2d),where=r_2d>1e-8)
field_theta_approx = np.divide(dipole_l*np.sin(theta_2d),r_2d**3,out=np.zeros_like(theta_2d),where=r_2d>1e-8)

# Transform to Cartesian to plot
field_x_approx = np.sin(theta_2d)*field_r_approx + np.cos(theta_2d)*field_theta_approx
field_z_approx = np.cos(theta_2d)*field_r_approx - np.sin(theta_2d)*field_theta_approx

# Use potential as background
plt.imshow(pot_approx,vmin=-vlim,vmax=vlim,extent=(-10,10,-10,10),origin='lower')
plt.colorbar(label="Electrostatic potential")
plt.plot(pos_p[0],pos_p[1],'ro')
plt.plot(pos_m[0],pos_m[1],'bo')
plt.streamplot(x_2d,z_2d,field_x_approx,field_z_approx,density=1.0,color='white')
plt.axis('scaled')

How similar these will be depends on how close to the dipole we are: in the far field, they are almost identical, but in the near field they are very different.  The main areas of difference are near the dipole (where the approximation breaks down).